# Code tree traversal

## Node class

In [967]:
from typing import Iterator

class Tree:
    """A Tree is a collection of Nodes with cached traversal data.

    A Tree is initialized from a root Node, where upon it will perform a
    tree traversal to cache Node data and store the idxorder traversal. 
    This cache will be auto-updated if the tree topology is changed.

    Parameters
    ----------
    treenode: Node
        The root Node of a tree data structure of Nodes.

    Attributes
    ----------
    nnodes: int
        The number of total Nodes in the tree.
    ntips: int
        The number of tip Nodes in the tree.
    
    Examples
    --------
    >>> treenode = get_random_tree(ntips=20)
    >>> tree = Tree(treenode)
    >>> print(tree)
    """
    def __init__(self, treenode: "Node"):
        self.treenode = treenode

        # attributes to be filled: cached traversal data
        self.nnodes: int = 0
        self.ntips: int = 0
        self._idx_dict: Dict[int, 'Node'] = {}

        # run caching of Node data.
        self._update_cache()

    def __repr__(self) -> str:
        """Return a representation of the tree as a string."""
        tip_name_string = str([self[i].name for i in range(5)])[:20] + ", ...]"
        return f"<Tree ntips={self.ntips}, tips={tip_name_string}>"

    def __getitem__(self, idx: int) -> "Node":
        """Return a Node by its cached idxorder label (idx)"""
        return self._idx_dict[idx]  

    def __iter__(self) -> Iterator["Node"]:
        """Return an iterator over Nodes in idxorder."""
        return (self[i] for i in range(self.nnodes))

    def _update_cache(self) -> Iterator['Node']:
        """Yield Nodes in idxorder, fills ._idx_dict cache, and sets Node.idx.

        This function must be called anytime the tree structure is modified
        by including it at the end of a function that modifies the tree.
        """
        # reset values
        self.nnodes = 0
        self.ntips = 0
        self._idx_dict.clear()
        
        # iterate in idxorder to set Node.idx attributes and counters
        for idx, node in enumerate(self.traverse('idxorder')):
            node._idx = idx
            self._idx_dict[node._idx] = node
            self.nnodes += 1
            if node.is_leaf():
                self.ntips += 1

    def traverse(self, strategy: str="idxorder") -> Iterator['Node']:
        """See :meth:`Node.traverse` docstring."""
        return self.treenode.traverse(strategy)

    
Tree.traverse.__doc__ = Node.traverse.__doc__


In [971]:
%%timeit
[i for i in t.traverse("levelorder")]

27.9 µs ± 576 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [972]:
%%timeit
[i for i in t._idx_dict.values()]

4.5 µs ± 90.5 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


In [966]:
t = Tree(root_node)
t.traverse?

In [961]:
Tree(root_node).traverse?

Object `traverse` not found.


In [ ]:
Tree(root_node).traverse

In [ ]:
Tree(root_node).traverse

In [818]:

    from typing import Tuple, Optional, Iterator
    from collections import deque

    class Node:
        """A Node instance that can connect with other Nodes to form a Tree.

        Parameters
        ----------
        name: str
            A name string associated with a Node when printed or visualized.
        dist: float
            A float value as the distance between this Node and its parent (up)

        Attributes
        ----------
        children: Tuple
            A tuple of Node instances that are descended from this Node.
        up: Node or None
            A Node that is ancestral to this Node, or None if this Node is root.

        Examples
        --------
        >>> node = Node("A")
        >>> node.add_child("B")
        >>> print(node.name, node.children)
        """
        def __init__(self, name: str="", dist: float=0.):
            self.name = str(name)
            self.dist = float(dist)
            self.children: Tuple['Node'] = ()
            self.up: Optional['Node'] = None

        def __repr__(self) -> str:
            """Return string representation as Node(name)."""
            return f"Node({self.name})"

        def is_leaf(self) -> bool:
            """Return True if Node is a leaf (i.e., has no children)."""
            return not bool(self.children)

        def is_root(self) -> bool:
            """Return True if Node is the root (i.e., has no ancestor)."""
            return bool(self.up is None)

        def add_child(self, name: str="", dist: float=0.) -> "Node":
            """Add a Node as a child to this one."""
            new_node = Node(name=name, dist=dist)
            new_node.up = self
            self.children += (new_node,)
            return new_node

        ############################################################
        ## TRAVERSAL FUNCTIONS                                    ##
        ############################################################

        def traverse(self, strategy: str="levelorder") -> Iterator['Node']:
            """Return a generator over all Nodes using a tree traversal strategy.

            Parameters
            ----------
            strategy: str
                A traversal strategy for the order in which nodes will
                be visited: 'preorder', 'postorder', 'levelorder',
                'inorder', or 'idxorder'.

            Notes
            -----
            preorder:
                Parents are visited before children. Traverses all the way
                down each left subtree before backtracking to right subtree.
            postorder:
                Children are visited before parents. The left subtree is
                visited, then right, then the parent.
            levelorder:
                Nodes the same depth from root are visited left to
                right, before descending to next level.
            idxorder:
                Leaf nodes are visited left to right, followed by internal
                nodes in postorder traversal order.
            """
            if strategy == "preorder":
                return self._traverse_preorder()
            if strategy == "levelorder":
                return self._traverse_levelorder()
            if strategy == "postorder":
                return self._traverse_postorder()
            if strategy == "idxorder":
                return self._traverse_idxorder()
            raise ValueError(
                f"strategy {strategy} is not a supported traversal algorithm")

        def _traverse_preorder(self) -> Iterator['Node']:
            """Iterate over all nodes by 'preorder' traversal."""
            queue = deque([self])
            while queue:
                node = queue.pop()
                yield node
                queue.extend(node.children[::-1])

        def _traverse_levelorder(self) -> Iterator['Node']:
            """Iterate over all desdecendant nodes in levelorder."""
            queue = deque([self])
            while queue:
                node = queue.popleft()
                yield node
                queue.extend(node.children)

        def _traverse_postorder(self) -> Iterator['Node']:
            """Iterate over all descendant nodes in tip-to-root order."""
            queue = deque([self])
            stack = deque()
            while queue:
                # get node from queue and add to the output stack
                node = queue.pop()
                stack.append(node)
                # add node's children to the queue
                queue.extend(node.children)
            # yield nodes in reverse order they were added to stack
            while stack:
                yield stack.pop()

        def _traverse_idxorder(self) -> Iterator['Node']:
            """Iterates over all nodes in 'idx' order."""
            queue = deque([self])
            inner_stack = deque()
            outer_stack = deque()
            node = self
            while queue:

                # get node from end of the queue
                node = queue.pop()

                # if node is leaf insert to output stack ntips from right
                if node.is_leaf():
                    outer_stack.append(node)

                # if not leaf then insert to output stack...
                else:
                    inner_stack.append(node)

                # add node's children to the queue
                queue.extend(node.children)

            # return nodes in reverse order they were added to stack
            while outer_stack:
                yield outer_stack.pop()
            while inner_stack:
                yield inner_stack.pop()

        ######################################################
        ## LEAF AND NAME RETRIEVAL                          ##
        ######################################################

        def _iter_leaves(self) -> Iterator['Node']:
            """Return a Generator of leaves descended from this node in
            idxorder."""
            for node in self.traverse(strategy="idxorder"):
                if not node.children:
                    yield node

        def get_leaves(self) -> Tuple['Node']:
            """Return a list of leaf nodes descended from this node in
            idxorder."""
            return tuple(self._iter_leaves())

        def _iter_leaf_names(self) -> Iterator[str]:
            """Return a Generator of names of Nodes descended from this
            node in idxorder."""
            for node in self._iter_leaves():
                yield node.name

        def get_leaf_names(self) -> Tuple[str]:
            """Return a list of names of Nodes descended from this node
            in idxorder."""
            return tuple(self._iter_leaf_names())

        #################################################
        ## NODE RELATIVE RETRIEVAL / TRAVERSAL
        #################################################

        def _iter_sisters(self) -> Iterator['Node']:
            """Return a Generator to iterate over sister nodes."""
            if self.up:
                for child in self.up.children:
                    if child != self:
                        yield child

        def get_sisters(self) -> Tuple['Node']:
            """Return list of other Nodes that are children of same parent."""
            return tuple(self._iter_sisters())

        def _iter_descendants(self, strategy: str="levelorder") -> Iterator['Node']:
            """Return a Generator of descendant Nodes (not including self)."""
            for node in self.traverse(strategy=strategy):
                yield node

        def get_descendants(self, strategy: str="levelorder") -> Tuple['Node']:
            """Return a list of descendant Nodes (not including self)."""
            return tuple(self._iter_descendants(strategy=strategy))

        def _iter_ancestors(self) -> Iterator['Node']:
            """Return a Generator of Nodes on path from this node to root."""
            node = self
            while node.up:
                yield node

        def get_ancestors(self) -> Tuple['Node']:
            """Return a list of Nodes on path from this node to root."""
            return tuple(self._iter_ancestors())            


In [824]:
# get a random 10-tip tree of connected Nodes
root_node = get_random_tree(ntips=6, seed=123)

# print Nodes in each traversal order
for strategy in ["levelorder", "postorder", "preorder", "idxorder"]:
    order = root_node.traverse(strategy=strategy)
    print(strategy, list(order))

    
# example: print each Node and its descendants
for node in root_node.traverse("levelorder"):
    print(f"{node}:\ttips={node.get_leaf_names()}")

levelorder [Node(root), Node(left1), Node(right1), Node(left3), Node(right3), Node(left2), Node(right2), Node(left5), Node(right5), Node(left4), Node(right4)]
postorder [Node(left5), Node(right5), Node(left3), Node(left4), Node(right4), Node(right3), Node(left1), Node(left2), Node(right2), Node(right1), Node(root)]
preorder [Node(root), Node(left1), Node(left3), Node(left5), Node(right5), Node(right3), Node(left4), Node(right4), Node(right1), Node(left2), Node(right2)]
idxorder [Node(left5), Node(right5), Node(left4), Node(right4), Node(left2), Node(right2), Node(left3), Node(right3), Node(left1), Node(right1), Node(root)]
Node(root):	tips=('left5', 'right5', 'left4', 'right4', 'left2', 'right2')
Node(left1):	tips=('left5', 'right5', 'left4', 'right4')
Node(right1):	tips=('left2', 'right2')
Node(left3):	tips=('left5', 'right5')
Node(right3):	tips=('left4', 'right4')
Node(left2):	tips=('left2',)
Node(right2):	tips=('right2',)
Node(left5):	tips=('left5',)
Node(right5):	tips=('right5',)
N

In [808]:
from collections import deque
a = list(get_random_tree(5, seed=123)._traverse_postorder())
b = list(get_random_tree(5, seed=123)._traverse_postorder2())

In [815]:
t = get_random_tree(5)
t._traverse_levelorder()

TypeError: 'generator' object is not subscriptable

In [811]:
list(b)

[Node(left3),
 Node(left4),
 Node(right4),
 Node(right3),
 Node(left1),
 Node(left2),
 Node(right2),
 Node(right1),
 Node(root)]

## Generating trees

In [12]:
def get_ladder_tree(ntips: int) -> "Node":
    """Return a ladder-like tree of Node objects with 'ntips' tip Nodes.

    Parameters
    ----------
    ntips: int
        The number of tip Nodes that must exist before the tree is returned.

    Returns
    -------
    Node
        The root Node of the set of connected Nodes is returned.
    """
    # create root Node with name=root and store as current node variable
    node = root = Node(name="root", dist=0)

    for idx in range(0, ntips, 2):
        # add two children to the focal node
        child0 = node.add_child(name=str(idx), dist=1)
        child1 = node.add_child(name=str(idx + 1), dist=1)

        # make right child the new focal node
        node = child1
    return root

In [88]:
def get_ladder_tree(ntips: int) -> "Node":
    """Return a ladder-like tree of Node objects with 'ntips' tip Nodes.

    Parameters
    ----------
    ntips: int
        The number of tip Nodes that must exist before the tree is returned.

    Returns
    -------
    Node
        The root Node of the set of connected Nodes is returned.
    """
    # create root Node with name=root and store as current node variable
    node = root = Node(name="root", dist=0)

    # loop to add children until ntips is reached
    nnodes = 1
    while nnodes < ntips * 2 - 1:
        # add child to the current node variable
        child = node.add_child(name=str(nnodes), dist=1)
        
        # advance the counter of nnodes
        nnodes += 1
        
        # if this is the second child, then make it the new 'current' node.
        if len(node.children) == 2:
            node = child
    return root

In [197]:
def get_ladder_tree(ntips: int) -> "Node":
    """Return a ladder-like tree of Node objects with 'ntips' tip Nodes.

    Parameters
    ----------
    ntips: int
        The number of tip Nodes that must exist before the tree is returned.

    Returns
    -------
    Node
        The root Node of the set of connected Nodes is returned.
    """
    # create root Node with name=root and store as current node variable
    node = root = Node(name="root", dist=0)

    # add tip Nodes by splitting current 'node' into a bifurcation until ntips exist.
    for idx in range(1, ntips):
        child_l = node.add_child(name=f"{idx}-l", dist=1)
        child_r = node.add_child(name=f"{idx}-r", dist=1)        

        # make right child the new focal node
        node = child_r
    
    # return root Node that now has ntips descended tip Nodes
    return root

In [247]:
root = get_ladder_tree(5)

In [756]:
def recursive_count(node) -> int:
    """Return the number of Nodes descended from a Node object."""
    clens = sum(recursive_count(child) for child in node.children)
    print(node.name, "size=", 1 + clens)
    return 1 + clens
    
recursive_count(root)

left3 size= 1
left4 size= 1
right4 size= 1
right3 size= 3
left1 size= 5
left2 size= 1
right2 size= 1
right1 size= 3
root size= 9


9

In [755]:
def recursive_g(node) -> int:
    """Return the number of Nodes descended from a Node object."""
    nodes = [node]
    for child in node.children:
        nodes += recursive_g(child)
    return nodes

recursive_g(root)

[Node(root),
 Node(left1),
 Node(left3),
 Node(right3),
 Node(left4),
 Node(right4),
 Node(right1),
 Node(left2),
 Node(right2)]

In [218]:
def recursive_preorder_traversal(node):
    """Return Nodes in preorder traversal.
    
    Starts at the current (root) Node, then recursively traverses
    the right subtree then left subtree. The recursion calls this
    function, so that subtrees visit subtrees until a tip is reached.
    Tip Nodes return themselves, and internal Nodes return themselves
    plus the Nodes of their children, collected by recursion. Such
    that calling this function on the root returns all Nodes.
    """
    # tip Node: return Node in a list
    if not node.children:
        return [node]
    
    # internal Node: return [Node] + [left-child-descendants] + [right-child-descendants]
    nodes = [node]
    for child in node.children:
        nodes += recursive_preorder_traversal(child)
    return nodes

recursive_preorder_traversal(root)

[Node(root),
 Node(1-l),
 Node(1-r),
 Node(2-l),
 Node(2-r),
 Node(3-l),
 Node(3-r),
 Node(4-l),
 Node(4-r)]

In [519]:
tree = toytree.rtree.unittree(5)

In [559]:
from typing import Optional
import random

def get_random_tree(ntips: int, seed: Optional[int] = None) -> "Node":
    """Return a ladder-like tree of Node objects with 'ntips' tip Nodes.

    Parameters
    ----------
    ntips: int
        The number of tip Nodes that must exist before the tree is returned.

    Returns
    -------
    Node
        The root Node of the set of connected Nodes is returned.
    """
    # random seed
    random.seed(seed)

    # create root Node with name=root
    root = Node(name="root", dist=0)

    # store Nodes in a list that are currently tips
    tips = [root]

    # add tip Nodes by splitting 'node' into a bifurcation until ntips exist.
    for idx in range(1, ntips):

        # randomly sample one of the current tip Nodes
        node = random.choice(tips)
        tips.remove(node)

        # create two new tips from 'node' and add to tips list  
        child_l = node.add_child(name=f"left{idx}", dist=1)
        child_r = node.add_child(name=f"right{idx}", dist=1)
        tips.extend([child_l, child_r])

    # return root Node that now has ntips descended tip Nodes
    return root

get_random_tree(10)

Node(root)

In [ ]:
# Recursive function to perform preorder traversal on the tree
def preorder(root):
 
    # return if the current node is empty
    if root is None:
        return
 
    # Display the data part of the root (or current node)
    print(root.data, end=' ')
 
    # Traverse the left subtree
    preorder(root.left)
 
    # Traverse the right subtree
    preorder(root.right)

In [573]:
from typing import Iterator, List

In [583]:
node_a = Node("A")
node_a.add_child("B")
node_a.add_child("C")
print(node_a, node_a.children)

Node(A) (Node(B), Node(C))


In [586]:
toytree.tree("(a,b,c);").nnodes

4

In [582]:
def traverse_preorder_by_recursion(node) -> List['Node']:
    """Return a list of Nodes in 'preorder' traversal.

    This visits parents before children, by visiting all the way
    down the left and then right descendants of each node.
    """
    if not node.children:
        return [node]
    nodes = [node]
    for child in node.children:
        nodes += traverse_preorder_by_recursion(child)
    return nodes

def traverse_preorder_by_iteration(self) -> Iterator['Node']:
    """Yield Nodes in 'preorder' traversal order."""
    queue = [self]
    while queue:
        node = queue.pop()
        yield node
        queue.extend(node.children[::-1])

print(traverse_preorder_by_recursion(root))
print(list(traverse_preorder_by_iteration(root)))

[Node(root), Node(left1), Node(left3), Node(right3), Node(left4), Node(right4), Node(right1), Node(left2), Node(right2)]
[Node(root), Node(left1), Node(left3), Node(right3), Node(left4), Node(right4), Node(right1), Node(left2), Node(right2)]


In [754]:
def traverse_preorder_by_iteration(node) -> Iterator['Node']:
    """Yield Nodes in 'preorder' traversal order."""
    queue = [node]
    while queue:
        node = queue.pop()
        yield node
        queue.extend(node.children[::-1])
        
traverse_preorder_by_iteration(root_node)

<generator object traverse_preorder_by_iteration at 0x7f0166e91660>

In [752]:
def recursive_get_newick(node):
    """Return Nodes in preorder traversal.
    
    Starts at the current (root) Node, then recursively traverses
    the right subtree then left subtree. The recursion calls this
    function, so that subtrees visit subtrees until a tip is reached.
    Tip Nodes return themselves, and internal Nodes return themselves
    plus the Nodes of their children, collected by recursion. Such
    that calling this function on the root returns all Nodes.
    """
    # tip Node: return 'name'
    if not node.children:
        return node.name
    
    # internal Node: return (left-children, right-children)
    children = [recursive_get_newick(child) for child in node.children]
    
    # if root Node then return with ';' at end, else just return
    if node.is_root():
        return f"({','.join(children)});"
    else:
        return f"({','.join(children)})"

root = get_random_tree(5, seed=123)
new = recursive_get_newick(root)
new

'((left3,(left4,right4)),(left2,right2));'

In [751]:
root_node = get_random_tree(ntips=5, seed=123)
print(recursive_preorder_traversal(root_node))

[Node(root), Node(left1), Node(left3), Node(right3), Node(left4), Node(right4), Node(right1), Node(left2), Node(right2)]


In [749]:

def iterative_get_newick(node):
    """Return newick string of a tree using iterative traversal."""
    newick = []
    for n in node.traverse("postorder"):
        if n.is_leaf():
            newick.append(n.name)
        else:
            newick = [f"({','.join(newick)})"]
    return newick[0] + ";"
    

iterative_get_newick(tree)

'((((r0,r1),r2),r3,r4));'

In [746]:

def iterative_get_newick(node):
    """Return newick string of a tree using iterative traversal."""
    newick = []
    for n in node.traverse("preorder"):
        if not n.is_leaf():
            newick.append("")
        else:
            newick = [newick, n.name]
        print(n, newick)
    return newick[0]
    

iterative_get_newick(tree)

Node(idx=8, root) ['']
Node(idx=6, internal) ['', '']
Node(idx=5, internal) ['', '', '']
Node(idx=0, leaf) [['', '', ''], 'r0']
Node(idx=1, leaf) [[['', '', ''], 'r0'], 'r1']
Node(idx=2, leaf) [[[['', '', ''], 'r0'], 'r1'], 'r2']
Node(idx=7, internal) [[[['', '', ''], 'r0'], 'r1'], 'r2', '']
Node(idx=3, leaf) [[[[['', '', ''], 'r0'], 'r1'], 'r2', ''], 'r3']
Node(idx=4, leaf) [[[[[['', '', ''], 'r0'], 'r1'], 'r2', ''], 'r3'], 'r4']


[[[[['', '', ''], 'r0'], 'r1'], 'r2', ''], 'r3']

In [831]:
def iter_node_depths(node):
    """Yield the depths of Nodes in preorder traversal.

    """
    depths = {}
    for node in root_node.traverse("preorder"):
        if node.is_root():
            depths[node] = 0
        else:
            depths[node] = depths[node.up] + 1
    return depths

iter_node_depths(root_node)

{Node(root): 0,
 Node(left1): 1,
 Node(left3): 2,
 Node(left5): 3,
 Node(right5): 3,
 Node(right3): 2,
 Node(left4): 3,
 Node(right4): 3,
 Node(right1): 1,
 Node(left2): 2,
 Node(right2): 2}

In [851]:
a = Node("A")
a.add_child("B")
a.add_child("C")
a.children[1].add_child("D")
a.children[1].add_child("E")

#      A
#    B   C
#       D E

Node(E)

In [864]:
[i.name for i in toytree.tree("(B, (D, E));").traverse("postorder")]

['B', 'D', 'E', '', '']

In [856]:
def get_newick(root_node):
    """Return newick string of a tree using iterative traversal."""
    newick = []
    for node in root_node.traverse("postorder"):
        print(node)
        if node.is_leaf():
            newick.append(node.name)
            #print(newick)
        else:
            newick = [f"({','.join(newick)})"]
    return newick[0] + ";"


# example: create random tree of Nodes, get newick string, print.
root_node = get_random_tree(ntips=10, seed=123)
newick = get_newick(a)
print(newick)

Node(B)
Node(D)
Node(E)
Node(C)
Node(A)
((B,D,E));


In [890]:
def get_node_heights(root_node):
    """Return ...""" 
    # first pass, get edge distances from the root to each tip
    max_dist = 0
    for node in root_node.traverse("preorder"):
        if node.up:
            node.height += node.dist
        else:
            node.height = 0
        if node.is_leaf():
            max_height = max(max_dist, node.height)
    
    # second pass
        
dict(get_node_heights(t))

{Node(root): 2.0,
 Node(left1): 1.0,
 Node(left2): 0,
 Node(right2): 0,
 Node(right1): 1.0,
 Node(left3): 0,
 Node(right3): 1.0,
 Node(left4): 0,
 Node(right4): 0}

In [915]:
root_node = get_random_tree(ntips=100, seed=123)
print(root_node.get_leaf_names())

('left47', 'right47', 'left56', 'left75', 'right75', 'left52', 'right52', 'right46', 'left98', 'right98', 'left82', 'right82', 'left45', 'right45', 'left89', 'right89', 'left23', 'right23', 'left91', 'right91', 'right50', 'left96', 'right96', 'left76', 'right76', 'right73', 'left60', 'right60', 'right54', 'left51', 'right51', 'right21', 'left34', 'right34', 'left33', 'left90', 'right90', 'right64', 'left88', 'left97', 'right97', 'left72', 'right72', 'right48', 'left35', 'left67', 'right67', 'left71', 'right71', 'left74', 'right74', 'right53', 'right42', 'left86', 'right86', 'right29', 'left22', 'left84', 'right84', 'left85', 'right85', 'right80', 'right62', 'right27', 'left38', 'left41', 'right41', 'left13', 'left49', 'left61', 'left81', 'right81', 'left94', 'right94', 'right39', 'right28', 'left31', 'right31', 'left24', 'left78', 'left99', 'right99', 'left70', 'right70', 'left79', 'left93', 'left95', 'right95', 'right68', 'left69', 'left92', 'right92', 'left59', 'right59', 'left65', '

In [916]:
idx_dict = dict(enumerate(root_node.traverse("idxorder")))
idx_dict

{0: Node(left47),
 1: Node(right47),
 2: Node(left56),
 3: Node(left75),
 4: Node(right75),
 5: Node(left52),
 6: Node(right52),
 7: Node(right46),
 8: Node(left98),
 9: Node(right98),
 10: Node(left82),
 11: Node(right82),
 12: Node(left45),
 13: Node(right45),
 14: Node(left89),
 15: Node(right89),
 16: Node(left23),
 17: Node(right23),
 18: Node(left91),
 19: Node(right91),
 20: Node(right50),
 21: Node(left96),
 22: Node(right96),
 23: Node(left76),
 24: Node(right76),
 25: Node(right73),
 26: Node(left60),
 27: Node(right60),
 28: Node(right54),
 29: Node(left51),
 30: Node(right51),
 31: Node(right21),
 32: Node(left34),
 33: Node(right34),
 34: Node(left33),
 35: Node(left90),
 36: Node(right90),
 37: Node(right64),
 38: Node(left88),
 39: Node(left97),
 40: Node(right97),
 41: Node(left72),
 42: Node(right72),
 43: Node(right48),
 44: Node(left35),
 45: Node(left67),
 46: Node(right67),
 47: Node(left71),
 48: Node(right71),
 49: Node(left74),
 50: Node(right74),
 51: Node(righ

In [920]:
{'a'}.clear()

In [918]:
%%timeit
root_node.get_leaf_names()

68.3 µs ± 1.15 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [905]:
def iter_ndescendants_map(root_node) -> Iterator[Tuple['Node', int]]:
    """Yield a tuple with (Node, int)

    """ 
    for node in root_node.traverse("postorder"):
        if node.is_leaf():
            node.ndescendants = 0
        else:
            node.ndescendants = node.children[0].ndescendants + len(node.children)
        yield (node, node.ndescendants)

# example:
root_node = get_random_tree(ntips=6, seed=123)
dict(iter_ndescendants(root_node))

{Node(left5): 0,
 Node(right5): 0,
 Node(left3): 2,
 Node(left4): 0,
 Node(right4): 0,
 Node(right3): 2,
 Node(left1): 4,
 Node(left2): 0,
 Node(right2): 0,
 Node(right1): 2,
 Node(root): 6}

In [735]:
toytree.tree(new + ";").draw('s', layout='d');

<svg class="toyplot-canvas-Canvas" xmlns:toyplot="http://www.sandia.gov/toyplot" xmlns:xlink="http://www.w3.org/1999/xlink" xmlns="http://www.w3.org/2000/svg" width="350.0px" height="280.0px" viewBox="0 0 350.0 280.0" preserveAspectRatio="xMidYMid meet" style="background-color:transparent;border-color:#292724;border-style:none;border-width:1.0;fill:rgb(16.1%,15.3%,14.1%);fill-opacity:1.0;font-family:Helvetica;font-size:12px;opacity:1.0;stroke:rgb(16.1%,15.3%,14.1%);stroke-opacity:1.0;stroke-width:1.0" id="te19d4a11a7e44b2db373b8bd8a3bc26c"> 0 1 2 3 4 5 6 7 8 left3 left4 right4 left2 right2

In [294]:
# generate a random 8 tip ultrametric tree
tree = toytree.rtree.imbtree(ntips=5)

# draw facing downw with node index labels shown
c, a, m = tree.draw(ts='s', layout='d', node_labels=False, node_sizes=16, tip_labels=False);

import toyplot.svg
# toyplot.svg.render(c, "fig4.svg")

<svg class="toyplot-canvas-Canvas" xmlns:toyplot="http://www.sandia.gov/toyplot" xmlns:xlink="http://www.w3.org/1999/xlink" xmlns="http://www.w3.org/2000/svg" width="350.0px" height="250.0px" viewBox="0 0 350.0 250.0" preserveAspectRatio="xMidYMid meet" style="background-color:transparent;border-color:#292724;border-style:none;border-width:1.0;fill:rgb(16.1%,15.3%,14.1%);fill-opacity:1.0;font-family:Helvetica;font-size:12px;opacity:1.0;stroke:rgb(16.1%,15.3%,14.1%);stroke-opacity:1.0;stroke-width:1.0" id="te99ded973a7d4b1c93792fdb0320aab9">

### Animated tree traversal

In [530]:
import random

def get_random_tree(ntips: int, seed: Optional[int]=None) -> "Node":
    """Return a ladder-like tree of Node objects with 'ntips' tip Nodes.

    Parameters
    ----------
    ntips: int
        The number of tip Nodes that must exist before the tree is returned.

    Returns
    -------
    Node
        The root Node of the set of connected Nodes is returned.
    """
    # random seed
    random.seed(seed)
    
    # create root Node with name=root
    root = Node(name="root", dist=0)
    
    # store Nodes in a list that are currently tips
    tips = [root]

    # add tip Nodes by splitting 'node' into a bifurcation until ntips exist.
    for idx in range(1, ntips):
        
        # randomly sample one of the current tip Nodes
        node = random.choice(tips)
        tips.remove(node)
        
        # create two new tips from 'node' and add to tips list  
        child_l = node.add_child(name=f"right{idx}", dist=1)
        child_r = node.add_child(name=f"left{idx}", dist=1)
        tips.extend([child_l, child_r])

    # return root Node that now has ntips descended tip Nodes
    return root

In [531]:
root = get_random_tree(10, seed=123)
recursive_preorder_traversal(root)

[Node(root),
 Node(right1),
 Node(right3),
 Node(right5),
 Node(left5),
 Node(left3),
 Node(right4),
 Node(left4),
 Node(left1),
 Node(right2),
 Node(right6),
 Node(left6),
 Node(right7),
 Node(left7),
 Node(right9),
 Node(left9),
 Node(left2),
 Node(right8),
 Node(left8)]

In [379]:
tree = toytree.rtree.imbtree(5)
canvas = toyplot.Canvas(400, 300)
axes = canvas.cartesian(show=False)

m = tree.draw(axes=axes, layout='d', tip_labels=False)
coords = tree.get_node_coordinates(layout='d')
mark = axes.scatterplot(coords.x, coords.y, size=20)

<svg class="toyplot-canvas-Canvas" xmlns:toyplot="http://www.sandia.gov/toyplot" xmlns:xlink="http://www.w3.org/1999/xlink" xmlns="http://www.w3.org/2000/svg" width="400.0px" height="300.0px" viewBox="0 0 400.0 300.0" preserveAspectRatio="xMidYMid meet" style="background-color:transparent;border-color:#292724;border-style:none;border-width:1.0;fill:rgb(16.1%,15.3%,14.1%);fill-opacity:1.0;font-family:Helvetica;font-size:12px;opacity:1.0;stroke:rgb(16.1%,15.3%,14.1%);stroke-opacity:1.0;stroke-width:1.0" id="tee268626ba6446819323a7d4b6d2ff1a">

In [393]:
coords

,x,y
0,0.0000,0.00
1,1.0000,0.00
2,2.0000,0.00
3,3.0000,0.00
4,4.0000,0.00
5,3.5000,0.25
6,2.7500,0.50
7,1.8750,0.75
8,0.9375,1.00


[8, 0, 7, 1, 6, 2, 5, 3, 4]

In [436]:

tree = toytree.rtree.unittree(ntips=8, seed=123)

def plot_traversal_order(tree, strategy="postorder"):

    # draw the tree with nodes colored
    canvas, axes, mark = tree.draw(layout='d', tip_labels=False)
    canvas.text(canvas.width / 2, 20, f'"{strategy}" traversal', style={"font-size": "16px"})
    canvas.style["background-color"] = "white"

    # get node coordinates table
    coords = tree.get_node_coordinates(layout='d')

    # get node indices in specified traversal order
    nidxs = [i.idx for i in tree.traverse(strategy)]

    # create labeled markers in traversal order
    markers = [toyplot.marker.create(shape="o", label=str(idx)) for idx, nidx in enumerate(nidxs)]

    # get scatterplot Mark
    mark = axes.scatterplot(coords.x[nidxs], coords.y[nidxs], size=20, marker=markers)

    # iterate over each datum as a frame at 2 frames / second
    for frame in canvas.frames((coords.shape[0] + 1, 2)):

        # set opacity very low on all Nodes initially
        if frame.number == 0:
            for i in range(coords.shape[0]):
                frame.set_datum_style(mark, 0, i, style={"opacity":0.1})

        # increase opacity as each frame 
        else:
            frame.set_datum_style(mark, 0, frame.number - 1, style={"opacity":1.0})
    return canvas
    
    
for strategy in ["levelorder", "postorder", "preorder", "idxorder"]:
    canvas = plot_traversal_order(tree, strategy)
    toyplot.html.render(canvas, f"traversal-{strategy}-animated.html", style={"text-align": "center"})

In [426]:
toyplot.html.render(canvas, "/tmp/test.html", style={"text-align": "center", "width": "350px"},)

In [428]:
toyplot.html.render()

<module 'toyplot.config' from '/home/deren/miniconda3/lib/python3.9/site-packages/toyplot/config.py'>

In [375]:
frame.set_mark_style()
frame.set_datum_text()

<bound method Canvas.frame of <toyplot.canvas.Canvas object at 0x7f0168bc1a00>>

In [325]:
mark._table

x,y0,marker0,size0,fill0,stroke0,opacity0,title0,hyperlink0
-1.21584609844,1.76578887458,o,10,"(0.4, 0.7607843137254902, 0.6470588235294118, 1.0)","(0.4, 0.7607843137254902, 0.6470588235294118, 1.0)",1,None,None
-2.24429368366,-1.21353645934,o,10,"(0.4, 0.7607843137254902, 0.6470588235294118, 1.0)","(0.4, 0.7607843137254902, 0.6470588235294118, 1.0)",1,None,None
-0.780022786974,-0.19876074865,o,10,"(0.4, 0.7607843137254902, 0.6470588235294118, 1.0)","(0.4, 0.7607843137254902, 0.6470588235294118, 1.0)",1,None,None
1.12163425905,0.138574186948,o,10,"(0.4, 0.7607843137254902, 0.6470588235294118, 1.0)","(0.4, 0.7607843137254902, 0.6470588235294118, 1.0)",1,None,None
0.0884396790735,1.00182281796,o,10,"(0.4, 0.7607843137254902, 0.6470588235294118, 1.0)","(0.4, 0.7607843137254902, 0.6470588235294118, 1.0)",1,None,None
0.770628054746,-0.973718992953,o,10,"(0.4, 0.7607843137254902, 0.6470588235294118, 1.0)","(0.4, 0.7607843137254902, 0.6470588235294118, 1.0)",1,None,None
-0.106933380068,0.0632512911163,o,10,"(0.4, 0.7607843137254902, 0.6470588235294118, 1.0)","(0.4, 0.7607843137254902, 0.6470588235294118, 1.0)",1,None,None
1.17169120717,-1.87875890065,o,10,"(0.4, 0.7607843137254902, 0.6470588235294118, 1.0)","(0.4, 0.7607843137254902, 0.6470588235294118, 1.0)",1,None,None
1.24212604833,0.946671056942,o,10,"(0.4, 0.7607843137254902, 0.6470588235294118, 1.0)","(0.4, 0.7607843137254902, 0.6470588235294118, 1.0)",1,None,None
0.992415601414,-0.290165360181,o,10,"(0.4, 0.7607843137254902, 0.6470588235294118, 1.0)","(0.4, 0.7607843137254902, 0.6470588235294118, 1.0)",1,None,None


In [323]:
import numpy

x = numpy.random.normal(size=100)
y = numpy.random.normal(size=len(x))

canvas = toyplot.Canvas(300, 300)
axes = canvas.cartesian()
mark = axes.scatterplot(x, y, size=10)

for frame in canvas.frames(len(x) + 1):
    if frame.number == 0:
        for i in range(len(x)):
            frame.set_datum_style(mark, 0, i, style={"opacity":0.1})
    else:
        frame.set_datum_style(mark, 0, frame.number - 1, style={"opacity":1.0})

<svg class="toyplot-canvas-Canvas" xmlns:toyplot="http://www.sandia.gov/toyplot" xmlns:xlink="http://www.w3.org/1999/xlink" xmlns="http://www.w3.org/2000/svg" width="300.0px" height="300.0px" viewBox="0 0 300.0 300.0" preserveAspectRatio="xMidYMid meet" style="background-color:transparent;border-color:#292724;border-style:none;border-width:1.0;fill:rgb(16.1%,15.3%,14.1%);fill-opacity:1.0;font-family:Helvetica;font-size:12px;opacity:1.0;stroke:rgb(16.1%,15.3%,14.1%);stroke-opacity:1.0;stroke-width:1.0" id="t751e5d0db7b24de38fc7d8a7b028e8e6"> -2 -1 0 1 2 -3 -2 -1 0 1 2

In [313]:
def inorder(root):
    if root.children:
        # Traverse left
        inorder(root.children[0])
        # Traverse root
        yield root
        print(str(root) + "->", end='')
        # Traverse right
        inorder(root.children[1])
    else:
        yield root

In [314]:
list(inorder(root))

Node(root)->

[Node(root)]

In [87]:
def count(node):
    ntips = 0
    if node.children:
        for child in node.children:
            ntips += count(child)
    else:
        return 1
    return ntips
    
count(root)

12

In [66]:
list(range(0, 10, 2))

[0, 2, 4, 6, 8]

In [17]:
root = get_ladder_tree(9)
print(root)

Node(root)


In [15]:
import toytree
toytree.rtree.unittree(20).nnodes

39